In [42]:
import requests
import pandas as pd
import json 
from dateutil import parser

In [43]:
API_KEY= "7203735f4489a07b1fdaa82e0825b643-037a13c8ecd863561ee2fb38166bda77"
ACCOUNT_ID = "101-011-28651499-001"
OANDA_URL = "https://api-fxpractice.oanda.com/v3"

In [44]:
session = requests.Session()

In [45]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [46]:
params = dict(
    count = 10,
    granularity="H1",
    price="MBA"
)

In [47]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [48]:
response =session.get(url,params=None, data=None,headers=None)

In [49]:
response.status_code


200

In [50]:
data = response.json()

In [51]:
instruments_list = data['instruments']

In [52]:
len(instruments_list)

127

In [53]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [54]:
key_i = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision','marginRate']

In [55]:
instruments_dict = {}

In [56]:
for i in instruments_list: 
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_i}

In [57]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.0333'}

In [58]:
with open ("../data/instruments.json","w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [78]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity=granularity,
        price="MBA"
        )
    response =session.get(url,params=params, data=None,headers=None)
    data = response.json()

    if response.status_code == 200:
        if 'candles' not in data:
            data = []
        else:
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
        
    prices = ['mid','bid','ask']
    ohlc = ['o','h','l','c']   
    
    final_data = []
    for candle in data:
        if candle['complete']== False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']
        for p in prices:
            for o in ohlc:
                new_dict[f"{p}_{o}"]= float(candle[p][o])
        final_data.append(new_dict)
    df = pd.DataFrame.from_dict(final_data)
    return df

In [60]:
code, data = fetch_candles("EUR_USD", count=10,granularity="H4")

In [ ]:
code

In [ ]:
data[0]

In [71]:
candle

{'complete': False,
 'volume': 1369,
 'time': '2024-06-25T05:00:00.000000000Z',
 'bid': {'o': '1.07390', 'h': '1.07436', 'l': '1.07387', 'c': '1.07412'},
 'mid': {'o': '1.07397', 'h': '1.07444', 'l': '1.07394', 'c': '1.07420'},
 'ask': {'o': '1.07404', 'h': '1.07451', 'l': '1.07402', 'c': '1.07427'}}

In [70]:
candle[p]

{'o': '1.07404', 'h': '1.07451', 'l': '1.07402', 'c': '1.07427'}

In [77]:
df

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2024-06-24 10:00:00+00:00,3575,1.07226,1.07309,1.07196,1.07304,1.07219,1.07302,1.07189,1.07296,1.07233,1.07316,1.07204,1.07311
1,2024-06-24 11:00:00+00:00,3522,1.07302,1.07360,1.07266,1.07359,1.07295,1.07352,1.07258,1.07352,1.07309,1.07368,1.07273,1.07366
2,2024-06-24 12:00:00+00:00,4346,1.07358,1.07358,1.07262,1.07346,1.07351,1.07351,1.07255,1.07338,1.07365,1.07365,1.07270,1.07353
3,2024-06-24 13:00:00+00:00,3897,1.07347,1.07437,1.07346,1.07428,1.07340,1.07430,1.07338,1.07421,1.07354,1.07444,1.07353,1.07435
4,2024-06-24 14:00:00+00:00,4418,1.07426,1.07465,1.07272,1.07301,1.07419,1.07458,1.07264,1.07294,1.07433,1.07472,1.07279,1.07308
5,2024-06-24 15:00:00+00:00,2711,1.07300,1.07332,1.07234,1.07271,1.07292,1.07325,1.07227,1.07264,1.07307,1.07339,1.07241,1.07278
6,2024-06-24 16:00:00+00:00,1489,1.07268,1.07316,1.07261,1.07279,1.07261,1.07309,1.07254,1.07272,1.07275,1.07323,1.07268,1.07286
7,2024-06-24 17:00:00+00:00,1587,1.07278,1.07278,1.07220,1.07250,1.07270,1.07270,1.07213,1.07243,1.07285,1.07285,1.07227,1.07257
8,2024-06-24 18:00:00+00:00,1679,1.07250,1.07376,1.07244,1.07356,1.07243,1.07369,1.07237,1.07349,1.07258,1.07383,1.07252,1.07364
9,2024-06-24 19:00:00+00:00,1974,1.07354,1.07384,1.07343,1.07360,1.07346,1.07376,1.07336,1.07353,1.07361,1.07391,1.07350,1.07367
